In [1]:
from IPython.display import display, Markdown, Latex
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langgraph.graph import END,StateGraph
from typing_extensions import TypedDict
import os

In [6]:
# # Environment Variables
# os.environ['LANGCHAIN_TRACING_V2'] = 'true'
# os.environ["LANGCHAIN_PROJECT"] = "L3 Research Agent"

In [2]:
# Defining LLM
local_llm = 'llama3:8b'
llama3 = ChatOllama(model=local_llm, temperature=0)
llama3_json = ChatOllama(model=local_llm, format='json', temperature=0)

In [3]:
# Web Search Tool

wrapper = DuckDuckGoSearchAPIWrapper(max_results=10)
web_search_tool = DuckDuckGoSearchRun(api_wrapper=wrapper)

In [4]:
# Test Run
resp = web_search_tool.invoke("home depot news")
resp

'In an acceleration of its strategy, renovation giant Home Depot is expanding its business with professional builders as the home fixer-upper market stalls. For investors, this is good news, because it means Home Depot\'s attractive capital allocation policy will continue. In fiscal 2021, 2022, and 2023, the company paid a total of $23.2 billion in ... Home Depot announced its biggest deal ever to acquire SRS Distribution, a specialty distributor of supplies for home professionals. The retailer said the deal will boost its addressable market and help it win more business from contractors, roofers and landscapers. ATLANTA, March 28, 2024 /PRNewswire/ -- The Home Depot ®, the world\'s largest home improvement retailer, has entered into a definitive agreement to acquire SRS Distribution Inc. ("SRS"), a leading residential specialty trade distribution company across several verticals serving the professional roofer, landscaper and pool contractor. The Home Depot reports sales and earnings 

In [5]:
# Generation Prompt

generate_prompt = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an AI assistant for Research Question Tasks, that synthesizes web search results. 
    Strictly use the following pieces of web search context to answer the question. If you don't know the answer, just say that you don't know. 
    keep the answer concise, but provide all of the details you can in the form of a research report. 
    Only make direct references to material if provided in the context.
    
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    
    Question: {question} 
    Web Search Context: {context} 
    Answer: 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "context"],
)

# Chain
generate_chain = generate_prompt | llama3 | StrOutputParser()

In [6]:
# Test Run
question = "How are you?"
context = ""
generation = generate_chain.invoke({"context": context, "question": question})
print(generation)

I'm just an AI, I don't have feelings or emotions like humans do. I am functioning properly and ready to assist with your research question tasks. I don't have personal experiences or physical sensations, so I can't say how I am in the classical sense. However, I am designed to provide accurate and helpful responses to your questions, and I'm always happy to help!


In [7]:
# Router

router_prompt = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|>
    
    You are an expert at routing a user question to either the generation stage or web search. 
    Use the web search for questions that require more context for a better answer, or recent events.
    Otherwise, you can skip and go straight to the generation phase to respond.
    You do not need to be stringent with the keywords in the question related to these topics.
    Give a binary choice 'web_search' or 'generate' based on the question. 
    Return the JSON with a single key 'choice' with no premable or explanation. 
    
    Question to route: {question} 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    
    """,
    input_variables=["question"],
)

# Chain
question_router = router_prompt | llama3_json | JsonOutputParser()

# Test Run
question = "What's up?"
print(question_router.invoke({"question": question}))

{'choice': 'generate'}


In [8]:
# Query Transformation

query_prompt = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an expert at crafting web search queries for research questions.
    More often than not, a user will ask a basic question that they wish to learn more about, however it might not be in the best format. 
    Reword their query to be the most effective web search string possible.
    Return the JSON with a single key 'query' with no premable or explanation. 
    
    Question to transform: {question} 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    
    """,
    input_variables=["question"],
)

# Chain
query_chain = query_prompt | llama3_json | JsonOutputParser()

# Test Run
question = "What's happened recently with Macom?"
print(query_chain.invoke({"question": question}))

{'query': 'Macom recent news'}


In [9]:
# Graph State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        search_query: revised question for web search
        context: web_search result
    """
    question : str
    generation : str
    search_query : str
    context : str

# Node - Generate

def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    
    print("Step: Generating Final Response")
    question = state["question"]
    context = state["context"]

    # Answer Generation
    generation = generate_chain.invoke({"context": context, "question": question})
    return {"generation": generation}

# Node - Query Transformation

def transform_query(state):
    """
    Transform user question to web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended search query
    """
    
    print("Step: Optimizing Query for Web Search")
    question = state['question']
    gen_query = query_chain.invoke({"question": question})
    search_query = gen_query["query"]
    return {"search_query": search_query}


# Node - Web Search

def web_search(state):
    """
    Web search based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to context
    """

    search_query = state['search_query']
    print(f'Step: Searching the Web for: "{search_query}"')
    
    # Web search tool call
    search_result = web_search_tool.invoke(search_query)
    return {"context": search_result}


# Conditional Edge, Routing

def route_question(state):
    """
    route question to web search or generation.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("Step: Routing Query")
    question = state['question']
    output = question_router.invoke({"question": question})
    if output['choice'] == "web_search":
        print("Step: Routing Query to Web Search")
        return "websearch"
    elif output['choice'] == 'generate':
        print("Step: Routing Query to Generation")
        return "generate"

In [10]:
# Build the nodes
workflow = StateGraph(GraphState)
workflow.add_node("websearch", web_search)
workflow.add_node("transform_query", transform_query)
workflow.add_node("generate", generate)

# Build the edges
workflow.set_conditional_entry_point(
    route_question,
    {
        "websearch": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "websearch")
workflow.add_edge("websearch", "generate")
workflow.add_edge("generate", END)

# Compile the workflow
local_agent = workflow.compile()

In [11]:
def run_agent(query):
    output = local_agent.invoke({"question": query})
    print("=======")
    display(Markdown(output["generation"]))

In [12]:
run_agent("What's been up with Macom recently?")

Step: Routing Query
Step: Routing Query to Web Search
Step: Optimizing Query for Web Search
Step: Searching the Web for: "Macom recent news"
Step: Generating Final Response


Based on the provided web search context, here is a concise research report on what's been up with MACOM recently:

**Recent Acquisition:** MACOM Technology Solutions Holdings, Inc. (NASDAQ: MTSI) has completed its acquisition of the radio frequency business (RF Business) of Wolfspeed, Inc. on December 2, 2023. The RF Business is highly complementary to MACOM's portfolio and creates a compelling addition.

**Financial Results:** In February 2024, MACOM announced its financial results for its fiscal first quarter ended December 29, 2023, with revenue of $157.1 million, a decrease of 12.7% compared to the previous year. In May 2024, the company reported its financial results for its fiscal second quarter ended March 29, 2024, with revenue of $181.2 million, an increase of 7.0% compared to the previous year.

**Product Showcase:** MACOM will showcase its latest RF, microwave, and millimeter wave products at Booth #1234 during a trade show in June 2024.

**Other News:** In August 2023, Wolfspeed, Inc. announced that it had entered into a definitive agreement to sell its radio frequency business (Wolfspeed RF) to MACOM Technology Solutions Holdings, Inc. for approximately $75 million in cash and 711,528 shares of MACOM common stock.

Overall, recent news from MACOM includes the completion of its acquisition of Wolfspeed's RF Business, financial results for its fiscal first and second quarters, a product showcase at a trade show, and the sale of Wolfspeed's RF business to MACOM.